In [1]:
import pandas as pd

In [2]:
servers = pd.DataFrame()
servers['name'] = [f'PC{i:02}' for i in range(1, 5)] + \
                [f'PC{i:02}' for i in range(6, 8)] + \
                [f'PC{i:02}' for i in range(9, 11)]
servers['hostname'] = ['172.19.4.225', '172.19.4.204', '172.19.4.235',
                 '172.19.4.200', '172.19.4.215', '172.19.4.238',
                 '172.19.4.203', '172.19.4.234']
servers['username'] = ['robert'] + ['iapucp'] * 7
servers['password'] = ['291295'] + ['iapucp504'] * 5 + \
                      ['intart504'] + ['iapucp504']
servers['model'] = ['HP i7'] * 3 + ['MSI'] + ['HP i7'] + \
                   ['MSI'] * 2 + ['Lenovo i5']


In [3]:
servers

,name,hostname,username,password,model
0,PC01,172.19.4.225,robert,291295,HP i7
1,PC02,172.19.4.204,iapucp,iapucp504,HP i7
2,PC03,172.19.4.235,iapucp,iapucp504,HP i7
3,PC04,172.19.4.200,iapucp,iapucp504,MSI
4,PC06,172.19.4.215,iapucp,iapucp504,HP i7
5,PC07,172.19.4.238,iapucp,iapucp504,MSI
6,PC09,172.19.4.203,iapucp,intart504,MSI
7,PC10,172.19.4.234,iapucp,iapucp504,Lenovo i5


In [ ]:
cd Desktop/Experiments && bash setup.sh

In [62]:
servers.to_csv('servers.csv', index=False)

In [4]:
import paramiko
def connect(servers, name):
    server = servers[servers.name==name]
    params = server[['hostname', 'username', 'password']] \
                .iloc[0].to_dict()
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy()) 
    ssh.connect(**params)
    return ssh

In [7]:
ssh = connect(servers, 'PC01')
stdin, stdout, stderr = ssh.exec_command('ls Desktop/Experiments')
for line in stdout: print(line, end='') 

Bags
CTNNB1-Raw-BCR.txt
CTNNB1-Raw-CD40.txt
DataCV
Dockerfile
environment.yml
FeatRanking
run_experiment.py


In [67]:
%%time
for server in servers.name:
    ssh = connect(servers, server)
    ssh.exec_command('mkdir -p ~/Desktop/Experiments')

CPU times: user 248 ms, sys: 8 ms, total: 256 ms
Wall time: 2.46 s


In [37]:
%%time
for server in servers.name:
    ssh = connect(servers, server)
    ssh.exec_command('mkdir -p ~/Desktop/Experiments/Results')

CPU times: user 232 ms, sys: 20 ms, total: 252 ms
Wall time: 1.86 s


In [ ]:
import os
os.chdir('..')

In [5]:
from scp import SCPClient

In [15]:
%%time
for server in servers.name:
    ssh = connect(servers, server)
    scp = SCPClient(ssh.get_transport())
    scp.put(['environment.yml', 'setup.sh'], 
            remote_path='~/Desktop/Experiments')

CPU times: user 188 ms, sys: 24 ms, total: 212 ms
Wall time: 2.6 s


## Transport Bags and FeatRanking

In [18]:
from tqdm import tqdm_notebook

In [20]:
%%time
for server in tqdm_notebook(servers.name):
    ssh = connect(servers, server)
    scp = SCPClient(ssh.get_transport())
    scp.put([path + 'Bags', path + 'FeatRanking'], 
            remote_path='~/Desktop/Experiments', recursive=True)

HBox(children=(IntProgress(value=0, max=8), HTML(value='')))


CPU times: user 7.88 s, sys: 592 ms, total: 8.47 s
Wall time: 48.1 s


In [22]:
def progress(filename, size, sent):
    print("\r%s\'s progress: %.2f%% " % \
          (filename, float(sent)/float(size)*100), end='')

In [25]:
%%time
for server in servers.name:
    ssh = connect(servers, server)
    ssh.exec_command('mkdir -p ~/Desktop/Experiments/DataCV')

CPU times: user 252 ms, sys: 16 ms, total: 268 ms
Wall time: 1.84 s


In [26]:
path = '_data/genomics-data/DataCV/'

In [ ]:
%%time
for server in tqdm_notebook(servers.name[1:], total=len(servers)-1):
    print(server)
    ssh = connect(servers, server)
    scp = SCPClient(ssh.get_transport(), progress = progress)
    scp.put([path + 'Dc', path + 'Di', path + 'Raw'], 
            remote_path='~/Desktop/Experiments/DataCV', 
            recursive=True)

HBox(children=(IntProgress(value=0, max=7), HTML(value='')))

PC02
b'Erk_CD40(0.01).tst.mat''s progress: 100.00% PC03
b'Jnk_BCR(0.01).tst.mat''s progress: 61.63% 

## Transport script

In [38]:
%%time
for server in servers.name:
    ssh = connect(servers, server)
    scp = SCPClient(ssh.get_transport())
    scp.put(['run_experiment.py'], 
            remote_path='~/Desktop/Experiments')

CPU times: user 196 ms, sys: 52 ms, total: 248 ms
Wall time: 2.53 s


In [36]:
servers

,name,hostname,username,password,model
0,PC01,172.19.4.225,robert,291295,HP i7
1,PC02,172.19.4.204,iapucp,iapucp504,HP i7
2,PC03,172.19.4.235,iapucp,iapucp504,HP i7
3,PC04,172.19.4.200,iapucp,iapucp504,MSI
4,PC06,172.19.4.215,iapucp,iapucp504,HP i7
5,PC07,172.19.4.238,iapucp,iapucp504,MSI
6,PC09,172.19.4.203,iapucp,intart504,MSI
7,PC10,172.19.4.234,iapucp,iapucp504,Lenovo i5


In [ ]:
sudo apt-get install screen

In [ ]:
screen
source activate iapucp
cd Desktop/Experiments
python run_experiment.py IRF4 Di BCR